# Building a Report Writing Agentic System using Reflection
In this tutorial, we will build a report writing agent that utilizes reflection to iteratively improve its output. By leveraging reflection, the agent can critique its own work and refine it in subsequent iterations, leading to a higher-quality final report.

This approach is inspired by the concept of reflection in LLM agent building, where the agent observes its past outputs and assesses the quality of its actions to enhance future performance.

### Overview
Our report writing agent will:

1. Generate an initial report based on a user-provided topic.
2. Reflect on the generated report by critiquing its content, structure, and style.
3. Iterate the process by incorporating the reflections to improve the report.
4. Repeat the reflection and revision steps for a set number of iterations or until the report meets a desired quality threshold.

### Imports and Setup
We start by importing the necessary libraries and setting up the environment:



In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

from typing import Annotated, List
from typing_extensions import TypedDict

from dotenv import load_dotenv

_ = load_dotenv()

### Defining the Models
One interesting thing we can do in multi agent systems is to use different models for different agents. 

We use the `gpt-4o-mini` model from OpenAI for the report generation and the `gpt-4o` model from OpenAI for the reflection steps. 

The `gpt-4o-mini` model is cheaper and faster than the `gpt-4o` model, so it is a good choice for the report generation step, while the `gpt-4o` model is more accurate and capable for the reflection step.

In [2]:
# Report Generation Model
generation_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=1500)

# Reflection Model
reflection_llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=1000)

### Defining the Report Generation Step
We create a chain for generating the initial report:

In [3]:
#Prompt for Generation
generation_prompt = ChatPromptTemplate.from_messages([
    ("system", """
        You are a professional report writer specializing in comprehensive and informative reports.
        Generate a detailed report based on the user's topic.
        The report MUST include the following sections:
        - Executive Summary: A brief overview of the main points.
        - Introduction: Background information and scope of the report.
        - Applications: Specific applications of AI in the topic.
        - Benefits: Advantages of using AI in the topic.
        - Challenges: Obstacles and limitations.
        - Future Potential: Possible future developments.
        - Conclusion: Summary and key takeaways.
        For each section, provide specific examples and detailed explanations. Aim for a report length of approximately 2000 words.
    """),
    MessagesPlaceholder(variable_name="messages"),
])

# Bind the prompt to the LLM
generate_report = generation_prompt | generation_llm


### Defining the Reflection Step
We create a prompt template for reflecting on and critiquing the generated report:

In [4]:
# Prompt for Reflection
reflection_prompt = ChatPromptTemplate.from_template("""
    You are a critical editor reviewing a report.
    Focus on improving the report's:
    - Accuracy: Identify any factual errors and suggest corrections.
    - Clarity: Point out any confusing or unclear sections and suggest rewrites.
    - Depth: Indicate areas where more information or analysis is needed and specify what to add.
    - Completeness: Check if all necessary sections are present and if they adequately address the topic.
    - Citations (if applicable): Ensure proper citation formatting and suggest any missing citations.

    Provide specific suggestions for revision. Do not simply critique the report. Instead, provide actionable feedback that the AI can use to improve the report.

    Objective: {input}
    Report: {messages}
""")
# Bind the prompt to the LLM
reflect_on_report = reflection_prompt | reflection_llm


### Defining the State
We define the state that will be passed through the LangGraph nodes:

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


### Building the Graph
We create nodes for generation, reflection, and a loop to iterate over the process:


In [15]:
# Initialize the graph builder
builder = StateGraph(State)

# Generation Node
async def generation_node(state: State) -> State:
    generated_messages = [await generate_report.ainvoke(state["messages"])]
    return {
        "messages": generated_messages,
        "input": state.get("input", "")  # Preserve 'input'
    }

builder.add_node("generate", generation_node)

# Reflection Node
async def reflection_node(state: State) -> State:
    # Swap message roles for reflection
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    translated_messages = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]

    # Construct the dictionary for the replanner prompt
    prompt_input = {
        "input": state.get("input", ""),  # Original input
        "plan": state.get("plan", []),    # Current plan
        "past_steps": state.get("past_steps", []),  # Past steps history
        "messages": translated_messages  # Add 'messages'
    }

    res = await reflect_on_report.ainvoke(prompt_input)

    # Treat reflection as human feedback
    return {
        "messages": [HumanMessage(content=res.content)],
        "input": state.get("input", ""),  # Preserve 'input'
        "plan": state.get("plan", []),    # Preserve 'plan'
        "past_steps": state.get("past_steps", [])  # Preserve 'past_steps'
    }

builder.add_node("reflect", reflection_node)

# Define edges
builder.add_edge(START, "generate")

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.boolean import BooleanOutputParser

# Define a completion check LLM (you might want a cheaper/faster model)
completion_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
completion_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that determines if a report is complete. If the report is complete, respond with 'YES'. If the report requires further refinement, respond with 'NO'."),
    ("user", "Here is the report: {report} Respond ONLY with 'YES' or 'NO', and nothing else."),
])
completion_parser = BooleanOutputParser()  # To get True/False output

completion_chain = completion_prompt | completion_llm | completion_parser

def should_continue(state: State):
    # Limit iterations to prevent infinite loops
    if len(state["messages"]) > 6:  # Adjust based on desired iterations
        print("Iteration limit reached.")
        return END

    # Get the latest report
    latest_report = state["messages"][-1].content  # Assuming the last message is the report

    # Check if the report is complete
    is_complete = completion_chain.invoke({"report": latest_report})

    if is_complete:
        print("Report is complete.")
        return END
    else:
        print("Report requires further refinement.")
        return "reflect"

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")

# Compile the graph with memory checkpointing
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)


In [ ]:
from IPython.display import display, Image

# Visualize the chatbot's workflow
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass  # Visualization requires additional dependencies

### Running the Agent
We run the agent with a user-provided topic:


In [16]:
config = {"configurable": {"thread_id": "1"}}
topic = "The Impact of Artificial Intelligence on Cyber Security"


In [17]:
async def run_agent():
    async for event in graph.astream(
        {
            "messages": [
                HumanMessage(content=topic)
            ],
        },
        config,
    ):
        if "generate" in event:
            print("=== Generated Report ===")
            print(event["generate"]["messages"][-1].content)
            print("\n")
        elif "reflect" in event:
            print("=== Reflection ===")
            print(event["reflect"]["messages"][-1].content)
            print("\n")

import asyncio
await run_agent()


Report requires further refinement.
=== Generated Report ===
# The Impact of Artificial Intelligence on Cyber Security

## Executive Summary

This report explores the significant impact of Artificial Intelligence (AI) on cyber security, delineating its applications, benefits, challenges, and future potential. AI technologies, such as machine learning and natural language processing, have revolutionized the way organizations protect their digital assets and respond to cyber threats. The integration of AI in cyber security enhances threat detection, response capabilities, and overall system resilience. However, the adoption of AI also introduces challenges, including the necessity for robust training data, potential biases in algorithms, and the risk of adversarial attacks. As organizations continue to embrace AI in their cyber security strategies, understanding its implications becomes increasingly critical.

## Introduction

In recent years, the proliferation of digital technologies ha

In [18]:
from IPython.display import display, Markdown
import asyncio

async def run_agent():
    final_report_markdown = ""  # Variable to store the final report in Markdown
    iteration = 1
    async for event in graph.astream(
        {
            "messages": [
                HumanMessage(content=topic)
            ],
        },
        config,
    ):
        print(f"=== Iteration {iteration} ===")
        if "generate" in event:
            print("--- Generating Report ---")
            display(Markdown(event["generate"]["messages"][-1].content))
            print("\n")
        elif "reflect" in event:
            print("--- Reflection ---")
            display(Markdown(event["reflect"]["messages"][-1].content))
            print("\n")
        # Check if we have a final response
        if "response" in event:
            print("=== Final Report ===")
            final_report_markdown = event["response"]
            display(Markdown(final_report_markdown))
            break  # Exit the loop after displaying the final report
        iteration += 1

    # Check if a final report was generated
    if final_report_markdown:
        print("\n--- Final Report (Stored in variable) ---")
        print(final_report_markdown)

        # Example: Writing the markdown to a file
        with open("final_report.md", "w") as f:
            f.write(final_report_markdown)
    else:
        print("\n--- No Final Report Generated ---")
        print("The iteration limit was reached before a final report was produced.")

### Conclusion
Congratulations! You've built a report writing agent that leverages reflection to iteratively improve its output. This agent can be a valuable tool for generating high-quality reports on various topics, with the ability to self-critique and refine its work.

Feel free to experiment further by:

- Changing the report topic.
- Modifying the prompts for generation and reflection.
- Integrating external data sources for up-to-date information.
- Adjusting the number of iterations or adding a quality threshold for termination.
